In [61]:
import cv2
import numpy as np

In [62]:
# Load the YOLOv3 model
model_weights = 'D:\Project\yolov3.weights'
model_config = 'yolov3.cfg'
net = cv2.dnn.readNetFromDarknet(model_config, model_weights)

In [63]:
# Load the class labels
class_labels = 'class_labels.txt'
with open(class_labels, 'r') as f:
    classes = f.read().splitlines()

In [67]:
# Load the input image
image = cv2.imread('awe1.jpg')

In [68]:
# Check the image size and resize if necessary
input_size = (416, 416)
if image.shape[:2] != input_size:
    image = cv2.resize(image, input_size)

In [69]:
# Get the model's output layer names
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

In [70]:
# Preprocess the image
blob = cv2.dnn.blobFromImage(image, 1/255.0, input_size, swapRB=True, crop=False)

In [71]:
# Set the input blob for the network
net.setInput(blob)

In [72]:
# Forward pass through the network
outputs = net.forward(output_layers)

In [79]:
# Adjust the confidence threshold
confidence_threshold = 500

In [80]:
# Process the detections
for output in outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        
        if confidence > confidence_threshold:
            # Get the bounding box coordinates
            center_x = int(detection[0] * image.shape[1])
            center_y = int(detection[1] * image.shape[0])
            width = int(detection[2] * image.shape[1])
            height = int(detection[3] * image.shape[0])
            
            # Calculate the top-left corner of the bounding box
            x = int(center_x - width / 2)
            y = int(center_y - height / 2)
            
            # Draw the bounding box and label on the image
            cv2.rectangle(image, (x, y), (x + width, y + height), (0, 255, 0), 2)
            label = f'{classes[class_id]}: {confidence:.2f}'
            cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


In [81]:
# Display the output image
cv2.imshow('Object Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [89]:
import cv2
import numpy as np

# Load the pre-trained YOLO weights and configuration
net = cv2.dnn.readNet("D:\Project\yolov3.weights", "D:\Project\yolov3.cfg")

In [91]:
# Load the class labels
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [92]:
# Set the confidence threshold and non-maximum suppression threshold
conf_threshold = 0.5
nms_threshold = 0.4

In [93]:
# Load the input image
image = cv2.imread("q1.jpg")

In [94]:
# Create a blob from the image to pass through the network
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)

In [95]:
# Set the input blob for the network
net.setInput(blob)

In [96]:
# Forward pass through the network
outputs = net.forward(net.getUnconnectedOutLayersNames())

In [97]:
# Initialize lists to store the detected class labels, confidences, and bounding box coordinates
class_ids = []
confidences = []
boxes = []

In [98]:

# Process each output layer
for output in outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        # Filter out weak detections
        if confidence > conf_threshold:
            # Scale the bounding box coordinates to the input image size
            box = detection[0:4] * np.array([image.shape[1], image.shape[0], image.shape[1], image.shape[0]])
            (center_x, center_y, width, height) = box.astype("int")

            # Calculate the top-left corner coordinates of the bounding box
            x = int(center_x - (width / 2))
            y = int(center_y - (height / 2))

            # Add the class label, confidence, and bounding box coordinates to the respective lists
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, int(width), int(height)])


In [99]:
# Apply non-maximum suppression to remove overlapping bounding boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

In [101]:
# Iterate over the remaining bounding boxes after suppression and draw them on the image
for i in indices:
    i = i
    box = boxes[i]
    x, y, width, height = box
    label = classes[class_ids[i]]
    confidence = confidences[i]

    # Draw the bounding box and label on the image
    cv2.rectangle(image, (x, y), (x + width, y + height), (0, 255, 0), 2)
    text = f"{label}: {confidence:.2f}"
    cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

In [102]:
# Display the image with the detected parts of the motorcycle
cv2.imshow("Motorcycle Parts Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()